In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, MaxPooling2D, AveragePooling2D, Activation, GRU, LSTM, Embedding, Conv1D
from tensorflow.keras.callbacks import EarlyStopping

In [41]:
df = pd.read_csv('rna.csv', index_col=0)
df['Encoded'] = df['Sequence'].apply(lambda seq: [*map(lambda c: 'agcu'.index(c) + 1, seq)])

In [42]:
X = np.array(df['Encoded'].to_list())
y = df['Activity'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1111111111111)

In [43]:
model = keras.Sequential([
    # Conv1D(16, 4, activation='relu', padding='same', input_shape=X.shape[1:]),
    # Conv1D(32, 4, activation='relu', padding='same'),
    Dense(32, activation='relu', input_shape=X.shape[1:]),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')    
])

model.compile(loss='mse', optimizer='adam')

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 32)                640       
_________________________________________________________________
dense_27 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_28 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_29 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 33        
Total params: 3,841
Trainable params: 3,841
Non-trainable params: 0
_________________________________________________________________


In [44]:
earlystopping = EarlyStopping(min_delta=0.0003, patience=4, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
61/61 [==============================] - 3s 44ms/step - loss: 0.0228 - val_loss: 0.0208
Epoch 2/100
61/61 [==============================] - 2s 30ms/step - loss: 0.0220 - val_loss: 0.0195
Epoch 3/100
61/61 [==============================] - 2s 31ms/step - loss: 0.0209 - val_loss: 0.0194
Epoch 4/100
61/61 [==============================] - 2s 33ms/step - loss: 0.0215 - val_loss: 0.0198
Epoch 5/100
61/61 [==============================] - 2s 35ms/step - loss: 0.0208 - val_loss: 0.0196
Epoch 6/100
61/61 [==============================] - 2s 32ms/step - loss: 0.0191 - val_loss: 0.0200


In [49]:
y_pred = model.predict(X_test).ravel()
np.corrcoef(y_pred, y_test)

array([[1.        , 0.19683617],
       [0.19683617, 1.        ]])